In [2]:
import cv2
import os
import math
import mmcv
import pandas as pd
import torch
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO
from ultralytics.yolo.utils.plotting import Annotator

from mmpose.apis import inference_top_down_pose_model, init_pose_model, vis_pose_result
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


%run utils/superimposition/functions.ipynb
%run utils/extraction/object_tracking.ipynb

In [3]:
class Configuration:
    def __init__(self, main_img, seg_net, video_format, fps, file_name):
        self.main_img = main_img
        self.seg_net = seg_net
        self.fourcc = cv2.VideoWriter_fourcc(*video_format)
        self.fps = fps
        self.main_path = f"videos/{file_name}/"
        self.processed_path = self.main_path + "processed/"
        self.csv_path = self.processed_path + f"{file_name}.csv"
        self.pose_config = "utils/skeleton/hrnet_w32_coco_256x192.py"
        self.pose_checkpoint = "https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth"

# List available video files
file_list = os.listdir("videos")
selected_file = file_list[8]

print(file_list, "\nSelected File:", selected_file)

# Create a configuration object
config = Configuration(main_img="RGB", seg_net="utils/segmentation/yolov8n-seg.pt", video_format="mp4v", fps=10, file_name=selected_file)

['2022-12-01_18-03-33', '2022-12-02_13-05-04', '2023-01-24_15-28-00', '2023-01-28_15-50-17', '2023-04-18_16-00-58', '2023-04-18_16-08-43', '2023-04-18_16-24-54', '2023-04-27_12-30-56', '2023-04-27_12-35-33', 'combined_outisde_13.gif', 'newStereo', 'outside_test', 'SII'] 
Selected File: 2023-04-27_12-35-33


In [4]:
# Initialize the segmentation and pose models
seg_model = YOLO(config.seg_net)
pose_model = init_pose_model(config.pose_config, config.pose_checkpoint, device)

# Create an object handler and a dictionary for holding images
handler = ObjectHandler()
images = {"RGB": None, "Thermal": None, "Depth": None}

# Set the input video path
path = f"{config.processed_path}{config.main_img}_sii.mp4"

# Open the input videos
rgb_video = cv2.VideoCapture(f"{config.processed_path}RGB_sii.mp4")
thermal_video = cv2.VideoCapture(f"{config.processed_path}Thermal_sii.mp4")
depth_video = cv2.VideoCapture(f"{config.processed_path}Depth_sii.mp4")

# Initialize the output video writer
track_out = cv2.VideoWriter(f"{config.processed_path}track.mp4", config.fourcc, config.fps, (512, 512))
wo_skel = cv2.VideoWriter(f"{config.processed_path}wo_skel.mp4", config.fourcc, config.fps, (512, 512))
skeleton = cv2.VideoWriter(f"{config.processed_path}skel.mp4", config.fourcc, config.fps, (512, 512))

frame_count = 0
# Process the input videos frame by frame
for result in seg_model.track(source=path, stream=True, verbose=False, classes=[0, 2, 4], device=0):

    # Read the frames from input videos
    ret, images["RGB"] = rgb_video.read()
    ret1, images["Thermal"] = thermal_video.read()
    ret2, images["Depth"] = depth_video.read()
    if not (ret & ret1 & ret2):
        break

    # Process the depth image
    stereo_frame_mapped = cv2.normalize(images["Depth"], None, 255, 0, cv2.NORM_INF, cv2.CV_32F)
    stereo_frame_mapped = cv2.convertScaleAbs(stereo_frame_mapped)
    stereo_frame_mapped = cv2.cvtColor(stereo_frame_mapped, cv2.COLOR_BGR2GRAY)
    stereo_frame_mapped = cv2.equalizeHist(stereo_frame_mapped)
    stereo_frame_mapped = cv2.applyColorMap(stereo_frame_mapped, cv2.COLORMAP_JET)
    images["Color_Depth"] = stereo_frame_mapped

    # If bounding boxes are detected
    if result.boxes and type(result.boxes.id) != type(None):

        # Collect Properties of Objects
        classes = result.boxes.cls
        masks = result.masks.data.cpu()
        boxes_array = np.array(result.boxes.xyxy.cpu())
        track_ids = result.boxes.id.cpu()
        frame_num = np.ones_like(classes) * frame_count
        object_results = [{'class': cls.item(), 'track_id': track_id.item(), 'bbox': bbox, 'mask': mask, 'frame_num': frame} 
                          for cls, track_id, bbox, mask, frame in zip(classes, track_ids, boxes_array, masks, frame_num)]

        # Apply masks to the images
        images["Superimposed"] = apply_mask(images.copy(), masks, config.main_img)

        # Perform pose estimation
        object_results = inference_top_down_pose_model(pose_model, images[config.main_img], object_results, format='xyxy')[0]

        # Add object results to the handler
        handler.append_objects(images, object_results)
        images["Superimposed_skel"] = vis_pose_result(pose_model, images["Superimposed"], object_results, bbox_color=(0, 0, 0))
        images["Skeleton"] = vis_pose_result(pose_model, np.zeros_like(images["Superimposed"]), object_results, bbox_color=(0, 0, 0))

        # Initialize annotator for visualizing the results
        annotator = Annotator(images["Superimposed_skel"])

        # Annotate the results with bounding boxes and track IDs
        for obj in object_results:
            color = handler.object_container[obj["track_id"]].color_id
            annotator.box_label(obj["bbox"], label=f'{obj["track_id"]}', color=color)
            
        # Get the final annotated frame
        frame = annotator.result()

        # Display the images
        for label in images:
            cv2.imshow(label, images[label])

        # Display the YOLO result
        cv2.imshow("YOLO", frame)

        # Write the output frame to the video
        track_out.write(images["Superimposed_skel"])
        wo_skel.write(images["Superimposed"])
        skeleton.write(images["Skeleton"])

        # Increment the frame counter
        frame_count += 1

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

            
# Release the output video and close all windows
track_out.release()
wo_skel.release()
skeleton.release()
cv2.destroyAllWindows()

load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth


In [18]:
csv=handler.create_dataframe()
csv.to_csv(config.csv_path, index=False)

In [19]:
display(csv)

,frame_num,track_id,class,bbox,color,temperature,spatials,keypoints
0,0.0,1,0,"[220.51303, 214.62631, 274.22693, 349.16418]","[45, 42, 36]",72.981556,"[-0.16669620806017416, -0.5001891293406483, 6....","[[259.8515, 236.56757, 0.71979], [259.8515, 23..."
1,0.0,2,0,"[427.98157, 215.84985, 473.6348, 335.2556]","[13, 10, 5]",86.645182,"[10.964271507076134, -1.1004748529828283, 19.8...","[[453.72333, 229.49289, 0.9116149], [456.05548..."
2,0.0,3,0,"[312.32257, 229.03647, 348.07202, 318.4135]","[34, 32, 25]",83.089521,"[1.8532553415304218, -0.44272375667183705, 8.8...","[[341.1076, 238.37564, 0.8279964], [341.1076, ..."
3,0.0,4,0,"[64.09203, 214.70319, 113.01798, 333.40192]","[3, 1, 0]",65.474632,"[-4.51692729328551, -0.48697820422672805, 9.52...","[[81.02042, 227.10626, 0.90686464], [83.33876,..."
4,0.0,5,2,"[13.793884, 221.65851, 137.17516, 281.93652]","[65, 61, 42]",78.143696,"[-4.92962883028065, 0.11476404641595356, 9.647...","[[81.10736, 226.89636, 0.93920946], [82.71388,..."
...,...,...,...,...,...,...,...,...
45330,7986.0,1057,0,"[332.4987, 224.62343, 353.78345, 284.62277]","[4, 6, 7]",83.476468,"[7.226168176910388, 0.11417871997425365, 29.29...","[[349.29333, 234.99443, 0.5802802], [349.29333..."
45331,7986.0,1058,0,"[96.73657, 216.08878, 157.8916, 372.03766]","[11, 4, 0]",78.793904,"[-2.142826115459815, -0.6338136515217646, 5.88...","[[141.78198, 229.33835, 0.7915062], [141.78198..."
45332,7987.0,1054,0,"[455.27982, 227.35257, 485.36298, 288.88202]","[1, 2, 3]",99.667037,"[2.2126522914280153, -0.021859084669385933, 3....","[[478.43323, 235.58456, 0.38657904], [470.021,..."
45333,7987.0,1057,0,"[334.9259, 225.66354, 360.53738, 286.04065]","[5, 5, 6]",80.266169,"[7.606841517337869, 0.012264886882291333, 29.2...","[[353.92264, 234.9206, 0.7904031], [353.92264,..."


In [9]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

obj_spatials = np.array(handler.object_container[2].properties['spatials']).transpose(1,0)

ax = plt.figure().add_subplot(projection='3d')

x = obj_spatials[0]
y = obj_spatials[1]
z = obj_spatials[2]

ax.plot(x, y, z, label='parametric curve')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

ax.legend()

plt.show()